##### 镜像构建脚本

In [2]:
import docker
import json
import os
client = docker.from_env()
REGISTRY = "registry.cn-shenzhen.aliyuncs.com/bwin/"
def login(username="lihong@namiao",password="BpmPortal123",registry="registry.cn-shenzhen.aliyuncs.com"):
    user = username
    pwd = password
    reg = registry
    print(user,pwd,reg)
    login = client.login(username=user,password=pwd,registry=reg)
    print (login)
    ok = login["Status"] == "Login Succeeded"
login() # login to aliyun registry
login(username="lihongwansui",password="Internet111",registry="https://index.docker.io/v1/") # login to docker hub

lihong@namiao BpmPortal123 registry.cn-shenzhen.aliyuncs.com
{'IdentityToken': '', 'Status': 'Login Succeeded'}
lihongwansui Internet111 https://index.docker.io/v1/
{'IdentityToken': '', 'Status': 'Login Succeeded'}


In [2]:
def load_versions(fp):
    with open('configration.json') as cfg_file:  
        cfg = json.load(cfg_file)
        return cfg
def update_versions(cfg):
    with open('configration.json', 'w') as outfile:  
        json.dump(cfg, outfile)
        
versions = load_versions()
if len(versions) == 0:
    cfg = {"1":1,"2":1,"3":1,"4":1,"5":1}
    update_versions(cfg)
    versions = load_versions()
    print(versions)

In [3]:
dirs =  [fn for fn in os.listdir('.')
              if os.path.isdir(fn) and os.path.exists(os.path.join(fn,"image.conf"))]
#print (dirs)
menus = {}
menu_idx = 1
for fn in dirs:
    with open(os.path.join(fn,'image.conf')) as cfg_file:  
        cfg = json.load(cfg_file)
        menus[str(menu_idx)] = ((cfg['image'],cfg['registry'],cfg['last_version'],fn))
    menu_idx += 1
for idx,de in menus.items():
    img,reg,ver,fn = de
    print (idx,":","[",img,"]",",当前版本:",ver)
    print (de)

1 : [ nginx lb for coin HK ] ,当前版本: 1.0
('nginx lb for coin HK', 'registry.cn-shenzhen.aliyuncs.com/bwin/back_service_4coin', '1.0', 'nginx_lb_coin')


In [ ]:
choice = input("选择要创建的镜像: ") 
if choice in menus:
    print("当前版本号:{}".format(menus[choice][2]))
else:
    print("选择错误！")
    #exit
version = input("输入新版本号:")


选择要创建的镜像: 1
当前版本号:1.0


In [83]:
def build_image(image_name,registry,tg,docker_path):
    build_ok = True
    try:
        print("开始创建-{}-镜像......".format(image_name))
        image = client.images.build(path=docker_path,tag="{}:{}".format(registry,tg),rm=True,pull=True,quiet=False)
        print("镜像构建完成 -({})".format(image.tag))
    except docker.errors.BuildError as be:
        build_ok = False
        print("镜像构建错误: {0}".format(be))
    except docker.errors.APIError as ae:
        build_ok = False
        print("镜像构建错误: {0}".format(ae))
    except TypeError as te:
        build_ok = False
        print("镜像构建错误: {0}".format(te))
    versions[1] = tg
    update_versions(versions)
    return build_ok

def run_task(name, registry, version, path):
    build_ok = build_image(name, registry, version, path)
    if build_ok:
        print("构建成功，准备push到镜像仓库......")
        try:
            for line in client.images.push(
                    repository=registry,
                    tag=version,
                    stream=True,
                    auth_config={
                        "username": "lihong@namiao",
                        "password": "BpmPortal123"
                    }):
                print(line)
            print("push 完成，请检查push结果。")
        except:
            print("push 失败！")

In [87]:
if choice == "1":
    build_ok = build_image("tomcat_7_jdk8","registry.cn-shenzhen.aliyuncs.com/bwin/tomcat_7_jdk8",version,"./tomcat_7_jdk8")
    if build_ok:
        print("构建成功，准备push到镜像仓库......")
        try:
            for line in client.images.push(repository="registry.cn-shenzhen.aliyuncs.com/bwin/tomcat_7_jdk8",tag=version,stream=True,auth_config={"username":"lihong@namiao","password":"BpmPortal123"}):
                print(line)
            print("push 完成，请检查push结果。")
        except:
            print("push 失败！")
if choice == "2":
    build_ok = build_image("nginx_tomcat_7_jdk8","registry.cn-shenzhen.aliyuncs.com/bwin/nginx_tomcat",version,"./nginx_tomcat_7_jdk8")
    if build_ok:
        print("构建成功，准备push到镜像仓库......")
        try:
            for line in client.images.push(repository="egistry.cn-shenzhen.aliyuncs.com/bwin/nginx_tomcat",tag=version,stream=True,auth_config={"username":"lihong@namiao","password":"BpmPortal123"}):
                print(line)
            print("push 完成，请检查push结果。")
        except:
            print("push 失败！")
if choice == "3":
    build_ok = build_image("front_service","registry.cn-shenzhen.aliyuncs.com/bwin/front_service",version,"./front_service")
    if build_ok:
        print("构建成功，准备push到镜像仓库......")
        try:
            for line in client.images.push(repository="egistry.cn-shenzhen.aliyuncs.com/bwin/front_service",tag=version,stream=True,auth_config={"username":"lihong@namiao","password":"BpmPortal123"}):
                print(line)
            print("push 完成，请检查push结果。")
        except:
            print("push 失败！")
if choice == "4":
    build_ok = build_image("trade_service","registry.cn-shenzhen.aliyuncs.com/bwin/trade_service",version,"./trade_service")
    if build_ok:
        print("构建成功，准备push到镜像仓库......")
        try:
            for line in client.images.push(repository="egistry.cn-shenzhen.aliyuncs.com/bwin/trade_service",tag=version,stream=True,auth_config={"username":"lihong@namiao","password":"BpmPortal123"}):
                print(line)
            print("push 完成，请检查push结果。")
        except:
            print("push 失败！")
if choice == "5":
    build_ok = build_image("wechat_service","registry.cn-shenzhen.aliyuncs.com/bwin/wechat_service",version,"./wechat_service")
    if build_ok:
        print("构建成功，准备push到镜像仓库......")
        try:
            for line in client.images.push(repository="egistry.cn-shenzhen.aliyuncs.com/bwin/wechat_service",tag=version,stream=True,auth_config={"username":"lihong@namiao","password":"BpmPortal123"}):
                print(line)
            print("push 完成，请检查push结果。")
        except:
            print("push 失败！")

开始创建-tomcat_7_jdk8-镜像......
镜像构建完成 -(<bound method Image.tag of <Image: 'tomcat_7_jdk8:latest', 'tomcat_7_jdk8:v_3.1', 'registry.cn-shenzhen.aliyuncs.com/bwin/tomcat_7_jdk8:3.1', 'registry.cn-shenzhen.aliyuncs.com/bwin/tomcat_7_jdk8:v_3.1'>>)
构建成功，准备push到镜像仓库......
b'{"status":"The push refers to a repository [registry.cn-shenzhen.aliyuncs.com/bwin/tomcat_7_jdk8]"}\r\n'
b'{"errorDetail":{"message":"Get https://registry.cn-shenzhen.aliyuncs.com/v2/: net/http: TLS handshake timeout"},"error":"Get https://registry.cn-shenzhen.aliyuncs.com/v2/: net/http: TLS handshake timeout"}\r\n'
push 完成，请检查push结果。
